<a href="https://colab.research.google.com/github/caesarcc/pucminas-tcc-fake-news-detection/blob/main/passo02_preparacao_do_PTT5_sumarizador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preparação do Modelo T5, pré-treinado em PT-BR, para sumarização

### Iniciando pela configuração do Google Colab, libs externas e acesso ao Google Drive

In [1]:
# lib para utilizar arquitetura transformers
!pip install -q transformers==4.18.0
# lib necessária para o tokenizador   
!pip install -q sentencepiece
# lib auxiliar para facilitar o scraping de artigos na web
!pip install -q newspaper3k

     |████████████████████████████████| 4.0 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 85.4 MB/s 
     |████████████████████████████████| 880 kB 58.1 MB/s 
     |████████████████████████████████| 86 kB 5.9 MB/s 
     |████████████████████████████████| 6.6 MB 55.4 MB/s 
     |████████████████████████████████| 1.2 MB 5.2 MB/s 
     |████████████████████████████████| 211 kB 5.1 MB/s 
     |████████████████████████████████| 81 kB 10.3 MB/s 
     |████████████████████████████████| 93 kB 2.2 MB/s 
     |████████████████████████████████| 7.4 MB 23.6 MB/s 


In [2]:
# Acesso ao Google Driver onde serão salvos os arquivos grandes
# Atenção: Todos arquivos usados também estão disponíveis nas pastas modelos e dados, respectivamente
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Importação de bibliotecas
import re
import newspaper
import torch
from transformers import T5Tokenizer
from transformers import T5Model, T5ForConditionalGeneration

### Carga dos modelos pré-treinados em arquitetura transformers, disponíveis na plataforma [Hugging Face](https://huggingface.co/models)

In [4]:
# Modelo trainado em PT-BR
model_name = 'unicamp-dl/ptt5-base-portuguese-vocab'
# Tokenizador para ptbr
tokenizer = T5Tokenizer.from_pretrained(model_name)
# Pesos do modelo em PyTorch
model_pt = T5ForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/738k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [30]:
artigo = newspaper.Article('https://www.bbc.com/portuguese/internacional-61818563', language='pt')
artigo.download()
artigo.parse()
#print(artigo.text[295:])

In [48]:
# Limpar textos
def limpar_textos(texto):
    # Corrige marcadores de nova linha indevidos (textuas e não textuais)
    texto = re.sub(r'(\n|\r)', ' ', texto)
    texto = re.sub('(\n|\r)', '', texto)
    # Remove urls no texto
    texto = re.sub(r'(https|http|ftp)?:\/\/(\w|\.|-|\/|\?|\=|\&|\%)*\b', '', texto)
    # Limpa restante do texto para manter apenas conteúdo útil
    texto = re.sub(u'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕçÇ:\-,." ]', '', texto)
    # Remove reticências e pontuação incorreta
    texto = texto.replace('...', '').replace("..", ".").replace(". .", ".")
    # Remove multiplos caracteres em branco
    texto = re.sub(r'\s+',' ', texto)
    return texto.strip()

texto = limpar_textos(artigo.text[295:])
qtde_tokens = len(re.findall(r"[\w']+|[.,!?;:/\"]", texto))
display(f"Quantidade de palavras: {qtde_tokens}", texto)

'Quantidade de palavras: 1162'

'Sem negar nem confirmar o teor de uma reportagem da agência Bloomberg, que afirmou que o presidente brasileiro Jair Bolsonaro PL teria pedido ao seu colega americano Joe Biden ajuda para se reeleger nas eleições presidenciais de outubro, a Casa Branca afirmou BBC News Brasil, por meio de um porta-voz, que espera que "os candidatos respeitem o resultado constitucional do processo eleitoral" e reafirmou "total confiança" no sistema de eleições do país. "Falando amplamente, temos total confiança no sistema eleitoral do Brasil. Em uma democracia consolidada como a brasileira, esperamos que os candidatos respeitem o resultado constitucional do processo eleitoral", afirmou um porta-voz da Casa Branca. A manifestação acontece menos de uma semana após o primeiro encontro entre Bolsonaro e Biden, em Los Angeles, s margens da 9a Cúpula das Américas, das quais os americanos eram anfitriões. Durante os primeiros dez minutos da reunião, abertos para a imprensa, o presidente americano fez questão d

### Codifica (Tokeniza) o texto de entrada

In [53]:
tokens_entrada = tokenizer.encode(texto, return_tensors="pt")
print(f"Quantidade de tokens gerados: {len(tokens_entrada[0])}")

Quantidade de tokens gerados: 1328


### Coloca o modelo e os tokens gerados no device disponível, usar Colab para GPU

In [50]:
# adiciona o modelo ao device disponível
device_disponivel = 'cuda' if torch.cuda.is_available() else 'cpu'
model_pt.to(device_disponivel)
tokens_entrada = tokens_entrada.to(device_disponivel)

### Gera a sumarização do texto com o modelo carregado

In [51]:
# max_length: O número máximo de tokens a serem gerado.
# length_penalty: Penalidade exponencial para o comprimento, 1,0 significa sem penalidade.
# num_beams: Especifica como usar busca heurística (beam search) em vez de busca gulosa (greedy search), 
#            O modelo tentará manter as 4 hipóteses mais prováveis ​​em cada passo de tempo.
# early_stopping: A geração seja concluída quando todas as hipóteses (beams) atingirem o final do token de string ( EOS ).
outputs = model_pt.generate(
    tokens_entrada, 
    max_length=400, 
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True)
display(len(outputs[0]), outputs)

400

tensor([[    0,  3647,  6955,    33,  1162, 16355,     9,  9258,     4,    17,
         14152,    11,  5797, 18259,  2702,     3,    13,  1670,    13,     9,
           409,   364, 13475, 15980,   310,   503,   989,  2621,    37,    45,
          6755,   574,  3853,  1287,   939,  1153,    20,    35,   115,  2406,
          1208,   117,  1364, 14368,     4,   312,     3,     7,   908,  5927,
          1670,  5184,  6612,   112,     3,    26,   386,     4,    16,  2215,
            14, 12045,     3,    13,  4185,    13,    15,   300,  4327, 16040,
          1954,  3633,     9,   849,  6950,    10,   519,  5720,    27,     8,
          3044,  9140,  6199,    15,   123,  1308,  7008,    27,    19,   276,
             4,  1364,    10,   221,     5,    15,  3528,   104,    59,  4103,
             3,  5428,   547,  7008,    19,   276,  5720,    10,   112,     5,
            40,    17,  8159,  7720,    58,    29,     7,   677,     3,  4185,
           573,    13,    30,  4327, 16040,  1954,  

### Decodifica tokens de resumo gerados com máximo de 400 palavras

In [54]:
texto_resumido = tokenizer.decode(outputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)

In [55]:
qtde_tokens = len(re.findall(r"[\w']+|[.,!?;:/\"]", texto_resumido))
# pode tranquilamente gerar menos de 400, só não pode gerar mais.
display(f"Quantidade de palavras: {qtde_tokens}", texto_resumido)

'Quantidade de palavras: 351'

'Sem negar nem confirmar o teor de uma reportagem da agência Bloomberg, que afirmou que o presidente brasileiro Jair Bolsonaro PL teria pedido ao seu colega americano Joe Biden ajuda para se reeleger nas eleições presidenciais de outubro, a Casa Branca afirmou BBC News Brasil, por meio de um porta-voz, que espera que "os candidatos respeitem o resultado constitucional do processo eleitoral" e reafirmou "total confiança" no sistema de eleições do país. "Falando amplamente, temos total confiança no sistema eleitoral do Brasil. Em uma democracia consolidada como a brasileira, esperamos que os candidatos respeitem o resultado constitucional do processo eleitoral", afirmou um porta-voz da Casa Branca. A manifestação acontece menos de uma semana após o primeiro encontro entre Bolsonaro e Biden, em Los Angeles, s margens da 9a Cúpula das Américas, das quais os americanos eram anfitriões. Durante os primeiros dez minutos da reunião, abertos para a imprensa, o presidente americano fez questão d

### Salva modelo no Google Drive para facilitar reuso (evitar novo download)

In [56]:
CAMINHO_MODELO = "drive/MyDrive/PUC/TCC/modelos/sumarizador_ptt5"
model_pt.save_pretrained(f"{CAMINHO_MODELO}")
tokenizer.save_pretrained(f"{CAMINHO_MODELO}")

('drive/MyDrive/PUC/TCC/modelos/sumarizador_ptt5/tokenizer_config.json',
 'drive/MyDrive/PUC/TCC/modelos/sumarizador_ptt5/special_tokens_map.json',
 'drive/MyDrive/PUC/TCC/modelos/sumarizador_ptt5/spiece.model',
 'drive/MyDrive/PUC/TCC/modelos/sumarizador_ptt5/added_tokens.json')